In [1]:
import json
import os
import pandas as pd

In [20]:
data_dir="C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/LWL & EF Tobii Data Export"

In [21]:
os.chdir(data_dir)

In [22]:
!dir

 Volume in drive C is Windows
 Volume Serial Number is 1E0A-261A

 Directory of C:\Users\tsb7592\Box\Baby Toolbox Operations\Domain Teams\Language\Tobii Pilot\LWL & EF Tobii Data Export

06/02/2021  09:10 PM    <DIR>          .
06/02/2021  09:10 PM    <DIR>          ..
06/02/2021  09:06 PM     (207,156,309) Baby Toolbox_Data_Export_06_02_2021.tsv
06/02/2021  09:06 PM    <DIR>          old data exports
06/02/2021  09:06 PM    <DIR>          Tobii AOIs
               1 File(s)    207,164,501 bytes
               4 Dir(s)  182,162,235,392 bytes free


In [23]:
tdf= pd.read_csv('Baby Toolbox_Data_Export_06_02_2021.tsv', sep='\t') 

In [24]:
tdf.shape

(305387, 380)

In [25]:
keep = [col for col in tdf.columns if 'AOI' in col]
keep = [col for col in keep if 'Calib' not in col]

In [27]:
k24 = keep[4:76]

In [28]:
print(k24)

['AOI[LeftStim_Shoulder]Hit', 'AOI[RightStim_Hose]Hit', 'AOI[WholeScreen]Hit.4', 'AOI[LeftStim_Ladder]Hit', 'AOI[RightStim_Melon]Hit', 'AOI[WholeScreen]Hit.5', 'AOI[LeftStim_Rooster]Hit', 'AOI[RightStim_Playground]Hit', 'AOI[WholeScreen]Hit.6', 'AOI[LeftStim_Bike]Hit', 'AOI[RightStim_Sneaker]Hit', 'AOI[WholeScreen]Hit.7', 'AOI[LeftStim]Hit', 'AOI[RightStim]Hit', 'AOI[WholeScreen]Hit.8', 'AOI[LeftStim]Hit.1', 'AOI[RightStim]Hit.1', 'AOI[WholeScreen]Hit.9', 'AOI[LeftStim]Hit.2', 'AOI[RightStim]Hit.2', 'AOI[WholeScreen]Hit.10', 'AOI[LeftStim]Hit.3', 'AOI[RightStim]Hit.3', 'AOI[WholeScreen]Hit.11', 'AOI[LeftStim]Hit.4', 'AOI[RightStim]Hit.4', 'AOI[WholeScreen]Hit.12', 'AOI[LeftStim]Hit.5', 'AOI[RightStim]Hit.5', 'AOI[WholeScreen]Hit.13', 'AOI[LeftStim]Hit.6', 'AOI[RightStim]Hit.6', 'AOI[WholeScreen]Hit.14', 'AOI[LeftStim]Hit.7', 'AOI[RightStim]Hit.7', 'AOI[WholeScreen]Hit.15', 'AOI[LeftStim]Hit.8', 'AOI[RightStim]Hit.8', 'AOI[WholeScreen]Hit.16', 'AOI[LeftStim]Hit.9', 'AOI[RightStim]Hit.9'

In [29]:
tdf=tdf[k24]

In [30]:
tdata = round(tdf.mean(),4).to_dict()

In [12]:
test = tdf[tdf['AOI[WholeScreen]Hit.4'].notnull()][['AOI[LeftStim_Shoulder]Hit',
 'AOI[RightStim_Hose]Hit',
 'AOI[WholeScreen]Hit.4']]

In [31]:
left =[]
right=[]
away =[]
onscreen = []
i = 0
for val in tdata:
    if i%3 == 0:
        rv = tdata[val]
        left.append(tdata[val])
    elif i%3 == 1:
        rv += tdata[val]
        right.append(tdata[val])
    else:
        rv = 1 - rv
        away.append(rv)
        onscreen.append(tdata[val])
    i+=1

In [32]:
k24[3:6]

['AOI[LeftStim_Ladder]Hit', 'AOI[RightStim_Melon]Hit', 'AOI[WholeScreen]Hit.5']

In [16]:
test['AOI[LeftStim_Shoulder]Hit'].sum()

541.0

# json file

In [37]:
data_dir="C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/Tobii Verbose Files/Claire"

In [38]:
os.chdir(data_dir)

In [20]:
newfile ='949BCA6A-D3D6-426C-9A94-E806F84E4E72@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00025@2021-05-11-22.21.08.674@ExecutiveFunction@Claire@VerboseGazeData.json'

In [21]:
#df=pd.read_json(newfile)

In [39]:
df=pd.read_json('949BCA6A-D3D6-426C-9A94-E806F84E4E72@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00024@2021-05-11-22.11.48.356@LookListening@Claire@VerboseGazeData.json')

In [40]:
cols=['itemID','eventName','gazeLocationName', 'gazeLocationOnScreen']

In [41]:
df=df[cols]

In [42]:
df.eventName.unique()

array(['beganCalibration', 'data', 'beganCalibrationRound1',
       'faceVerticesChanged', 'presentedCalibrationItem',
       'cameraImageCaptured', 'lookDetected', 'completedCalibrationItem',
       'calibrationAlignmentCheckPassed', 'completedCalibration',
       'beganLiveTest', 'presentedLiveItem', 'gazeLocationChanged',
       'audioStarted', 'audioCompleted', 'beganLookDetection',
       'endedLookDetection', 'completedLiveItem', 'presentedFillerScreen',
       'removedFillerScreen', 'faceLeftScene', 'faceReturnedToScene',
       'completedLiveTest', 'beganDriftPresentationRound',
       'presentedDriftCalibrationItem', 'completedDriftCalibrationItem',
       'completedDriftPresentationRound', 'instrumentCompleted'],
      dtype=object)

In [43]:
len(df.gazeLocationOnScreen.value_counts())

2

In [44]:
#Finding the index for presentLiveItem and completeLiveItem to subset the pieces we need for analysis
begin =list(df[df['eventName'].str.contains('presentedLiveItem')].index)

In [45]:
end = list(df[df['eventName'].str.contains('completedLiveItem')].index)

In [46]:
expected = pd.DataFrame(
    {'Left': left,
     'Right': right,
     'Away': away,'Onscreen':onscreen
    })
expected

,Left,Right,Away,Onscreen
0,0.6450,0.2454,0.1096,0.9459
1,0.1904,0.7676,0.0420,0.9730
2,0.6740,0.1871,0.1389,0.9396
3,0.6653,0.1346,0.2001,0.9462
4,0.2605,0.6518,0.0877,0.9280
5,0.2133,0.4963,0.2904,0.8860
6,0.2207,0.5573,0.2220,0.9511
7,0.5864,0.2306,0.1830,0.9013
8,0.5796,0.2903,0.1301,0.9447
9,0.2697,0.5603,0.1700,0.8930


In [47]:
len(end)

24

# Fisher Exact Test 

In [48]:
#tsv file
exp = pd.DataFrame(columns=('left', 'right','onScreenAway','offScreenAway'))
for i in range(24):
    lcol,rcol,wcol = k24[3*i:3*i+3]
    
    subset = tdf[tdf[wcol].notnull()]
    
    a=subset[lcol].sum()
    b=subset[rcol].sum()
    d=subset[(subset[lcol]==0) & (subset[rcol]==0) &(subset[wcol]==1)].shape[0]
    e=subset[(subset[lcol]==0) & (subset[rcol]==0) &(subset[wcol]==0)].shape[0]

    exp.loc[i] = [a, b ,d, e]

In [51]:
#json file
obs = pd.DataFrame(columns=('L', 'R','onScreenAway','offScreenAway'))
items =[]
for i in range(len(end)):
    sub = df.iloc[begin[i]+1:end[i]]
    items.extend(sub.itemID.dropna().unique())
    a, b, c = sub.gazeLocationName.value_counts()[['left','right','away']].round(4) 
    if len(sub[sub['gazeLocationName'] == 'away'].gazeLocationOnScreen.value_counts()) == 2:
        d,e = sub[sub['gazeLocationName'] == 'away'].gazeLocationOnScreen.value_counts()
    else:
        d = c
        e = 0
    obs.loc[i] = [a, b ,d, e]

In [53]:
exp

,left,right,onScreenAway,offScreenAway
0,1942.0,739.0,167.0,163.0
1,572.0,2306.0,45.0,81.0
2,1974.0,548.0,230.0,177.0
3,2002.0,405.0,440.0,162.0
4,763.0,1909.0,46.0,211.0
5,642.0,1494.0,531.0,343.0
6,664.0,1677.0,521.0,147.0
7,1676.0,659.0,241.0,282.0
8,1697.0,850.0,219.0,162.0
9,771.0,1602.0,180.0,306.0


In [52]:
obs

,L,R,onScreenAway,offScreenAway
0,314,84,83,0
1,109,371,2,0
2,293,159,20,9
3,377,55,33,15
4,162,262,45,13
5,285,100,88,6
6,138,308,27,7
7,320,147,13,1
8,150,277,33,20
9,270,137,66,8


In [79]:
exp.to_csv('C:/Users/TEMP/Box/Baby Toolbox Operations/Analysis/tsv_count.csv', index=False)

In [80]:
obs.to_csv('C:/Users/TEMP/Box/Baby Toolbox Operations/Analysis/json_count.csv', index=False)

In [72]:
l1 =obs.to_numpy()
l2= exp.to_numpy()

In [76]:
items

['Shoulder_Hose_G5',
 'Ladder_Melon_G5',
 'Rooster_Playground_G5',
 'Tricycle_Sneaker_G5',
 'Hose_Shoulder_G5',
 'Melon_Ladder_G5',
 'Playground_Rooster_G5',
 'Sneaker_Tricycle_G5',
 'Star_Moose_G6',
 'Turtle_Hammer_G6',
 'Cake_Bee_G6',
 'Lamp_Stove_G6',
 'Moose_Star_G6',
 'Hammer_Turtle_G6',
 'Bee_Cake_G6',
 'Stove_Lamp_G6',
 'Garden_Sweater_G7',
 'Ankle_Scissors_G7',
 'Woods_Tray_G7',
 'Bench_Rip_G7',
 'Sweater_Garden_G7',
 'Scissors_Ankle_G7',
 'Tray_Woods_G7',
 'Rip_Bench_G7']

In [77]:
import scipy.stats as stats
from scipy.stats import chi2_contingency
from scipy.stats import chi2

for i in range(24):
    data = [l1[i], l2[2]]
    
    stat, p, dof, expected = chi2_contingency(data)
    print(items[i])
    
    # interpret test-statistic
    prob = 0.95
    critical = chi2.ppf(prob, dof)
    print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
    if abs(stat) >= critical:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')

    # interpret p-value
    alpha = 1.0 - prob
    print('significance=%.3f, p=%.3f' % (alpha, p))
    if p <= alpha:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')
    print('')

Shoulder_Hose_G5
probability=0.950, critical=7.815, stat=205.763
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)

Ladder_Melon_G5
probability=0.950, critical=7.815, stat=268.473
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)

Rooster_Playground_G5
probability=0.950, critical=7.815, stat=55.133
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)

Tricycle_Sneaker_G5
probability=0.950, critical=7.815, stat=148.672
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)

Hose_Shoulder_G5
probability=0.950, critical=7.815, stat=136.355
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)

Melon_Ladder_G5
probability=0.950, critical=7.815, stat=183.149
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)

Playground_Rooster_G5
probability=0.950, critical=7.815, stat=173.240
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)

Sneaker_Tricycle_G5
prob

In [66]:
l2

array([[541., 341.,  95.,  65.],
       [176., 761.,   0.,  23.],
       [497., 314.,  20., 130.],
       [793., 129.,  48.,  71.],
       [357., 435.,  40., 128.],
       [471., 201., 151., 218.],
       [274., 594.,  91.,  81.],
       [600., 230.,  14., 117.],
       [353., 489.,  29.,  89.],
       [538., 198.,  11., 214.],
       [176., 824.,   0.,  41.],
       [381., 440.,  40., 180.],
       [681., 318.,  14.,  28.],
       [308., 483., 139., 110.],
       [706., 244.,  62.,  29.],
       [837.,  63.,   7.,  53.],
       [645., 289.,   0.,  28.],
       [673., 329.,  17.,  21.],
       [768., 194.,  26.,  52.],
       [461., 364.,   6., 130.],
       [543., 475.,   0.,  22.],
       [285., 619.,  56.,  80.],
       [336., 330.,  29., 265.],
       [725.,  70.,  27., 140.]])

In [142]:
from scipy.stats import chisquare
chisq = pd.DataFrame(columns=('items', 'stat','p'))
for i in range(24):
    ob = obs.iloc[i]
    ex = expected.iloc[i]
    stat, pv = chisquare(f_obs=ob, f_exp=ex)
    chisq.loc[i] = [items[i], round(stat,3) ,round(pv,3)]


In [144]:
chisq.to_csv('C:/Users/TEMP/Box/Baby Toolbox Operations/Analysis/chisquare.csv', index=False)